<a href="https://colab.research.google.com/github/Gau-6426/PCAP/blob/main/Code/segments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df_segments_status = pd.read_csv("https://raw.githubusercontent.com/Gau-6426/PCAP/main/Data/segment_status.csv")
print(df_segments_status.head(10))
df_segments_status.shape

   _id                updated_at  segment_id  velocity
0    0  2020-07-03T14:55:31.869Z       24845        20
1    1  2020-07-03T15:02:56.048Z       33923        10
2    2  2020-07-04T08:15:52.696Z       33824         5
3    3  2020-07-04T08:15:59.903Z       33824         5
4    4  2020-07-04T08:16:08.201Z       33824         5
5    5  2020-07-04T08:16:11.070Z       33824         5
6    6  2020-07-04T08:16:18.512Z       33824         5
7    7  2020-07-04T08:16:21.798Z       33824         5
8    8  2020-07-04T08:16:24.746Z       33824         5
9    9  2020-07-04T08:16:27.606Z       33824         5


(90938, 4)

In [5]:
df_segments = pd.read_csv("https://raw.githubusercontent.com/Gau-6426/PCAP/main/Data/segments.csv")
df_segments.head(10)


,_id,created_at,updated_at,s_node_id,e_node_id,length,street_id,max_velocity,street_level,street_name,street_type
0,0,2020-10-18T13:26:17.365Z,2020-10-18T13:26:17.365Z,373543511,5468660805,114,31096786,80.0,1,Quốc Lộ 1,trunk
1,1,2020-10-18T13:26:17.400Z,2020-10-18T13:26:17.400Z,5468660805,5738158916,9,31096786,80.0,1,Quốc Lộ 1,trunk
2,2,2020-10-18T13:26:17.435Z,2020-10-18T13:26:17.435Z,5738158916,5738158918,23,31096786,80.0,1,Quốc Lộ 1,trunk
3,3,2020-10-18T13:26:17.444Z,2020-10-18T13:26:17.444Z,5738158918,5738158912,66,31096786,80.0,1,Quốc Lộ 1,trunk
4,4,2020-10-18T13:26:17.452Z,2020-10-18T13:26:17.452Z,5738158912,5758104203,127,31096786,80.0,1,Quốc Lộ 1,trunk
5,5,2020-10-18T13:26:17.461Z,2020-10-18T13:26:17.461Z,5758104203,2242413001,22,31096786,80.0,1,Quốc Lộ 1,trunk
6,6,2020-10-18T13:26:17.468Z,2020-10-18T13:26:17.468Z,2242413001,4066528708,15,31096786,80.0,1,Quốc Lộ 1,trunk
7,7,2020-10-18T13:26:17.472Z,2020-10-18T13:26:17.472Z,4066528708,4066528709,15,31096786,80.0,1,Quốc Lộ 1,trunk
8,8,2020-10-18T13:26:17.477Z,2020-10-18T13:26:17.477Z,4066528709,4066528710,10,31096786,80.0,1,Quốc Lộ 1,trunk
9,9,2020-10-18T13:26:17.481Z,2020-10-18T13:26:17.481Z,4066528710,2242412994,13,31096786,80.0,1,Quốc Lộ 1,trunk


In [13]:
segment_sort=df_segments.sort_values(by='max_velocity')
segment_sort

,_id,created_at,updated_at,s_node_id,e_node_id,length,street_id,max_velocity,street_level,street_name,street_type
31005,31005,2020-10-18T13:27:51.399Z,2020-10-18T13:27:51.399Z,2072605243,2072605350,5,196966072,10.0,3,Đường dẫn ga Quốc nội,secondary_link
70399,70399,2020-10-18T13:29:45.939Z,2020-10-18T13:29:45.939Z,2072605183,2072605122,10,585907729,10.0,4,Đường dẫn ga Quốc nội,tertiary
49941,49941,2020-10-18T13:28:40.288Z,2021-04-04T16:33:58.937Z,2072605207,2072605062,12,326849839,10.0,3,Quận Tân Bình,secondary_link
70397,70397,2020-10-18T13:29:45.935Z,2020-10-18T13:29:45.935Z,2072605166,2072605372,17,585907729,10.0,4,Đường dẫn ga Quốc nội,tertiary
70396,70396,2020-10-18T13:29:45.933Z,2020-10-18T13:29:45.933Z,2072605147,2072605166,10,585907729,10.0,4,Đường dẫn ga Quốc nội,tertiary
...,...,...,...,...,...,...,...,...,...,...,...
84628,84628,2020-10-18T13:30:29.795Z,2020-10-18T13:30:29.795Z,5778600776,411925919,42,658328101,NaN,4,Võ Văn Tần,tertiary
84629,84629,2020-10-18T13:30:29.797Z,2020-10-18T13:30:29.797Z,411925919,3116310151,39,658328101,NaN,4,Võ Văn Tần,tertiary
84630,84630,2020-10-18T13:30:29.799Z,2020-10-18T13:30:29.799Z,3116310151,5778360106,22,658328101,NaN,4,Võ Văn Tần,tertiary
84631,84631,2020-10-18T13:30:29.802Z,2020-10-18T13:30:29.802Z,5778360106,5763168795,37,658328101,NaN,4,Võ Văn Tần,tertiary


In [14]:
segment_status_df = pd.read_csv("https://raw.githubusercontent.com/hlnynhi/PCAP/main/Traffic-data/segment_status.csv", index_col="_id", parse_dates=["updated_at"])
segment_df = pd.read_csv("https://raw.githubusercontent.com/hlnynhi/PCAP/main/Traffic-data/segments.csv", index_col="_id", parse_dates=["created_at", "updated_at"])

In [16]:
from math import ceil

def transform_LOS(segment_id, velocity):
    max_velocity = segment_df.loc[segment_id, "max_velocity"]
    if max_velocity is None:
        max_velocity = 50
    
    # Transform to label
    labels = ["A", "B", "C", "D", "E", "F"]
    threshold = 35
    if max_velocity >= 70:
        threshold = 45
    elif max_velocity >= 60:
        threshold = 40

    t = max(threshold - velocity, 0)
    return labels[min(ceil(t / 5), 5)]

def transform_report(row):
    """
    @Params:
        dt: Timestamp object of Pandas
    @Return:
        dict: {"date", "period_{hour}_{00|30}"}
    """
    LOS = transform_LOS(row["segment_id"], row["velocity"])
    dt = row["updated_at"]
    intervals = list(range(24))
    h = dt.hour
    m = "00" if dt.minute < 30 else "30"
    p_name = f"period_{h}_{m}"
    return dt.date(), dt.weekday(), p_name, LOS

In [19]:
dates = []
weekdays = []
p_names = []
LOSes = []

for _, row in segment_status_df.iterrows():
    date, weekday, p_name, LOS = transform_report(row)
    dates.append(date)
    weekdays.append(weekday)
    p_names.append(p_name)
    LOSes.append(LOS)

segment_status_df["date"] = dates
segment_status_df["weekday"] = weekdays
segment_status_df["period"] = p_names
segment_status_df["LOS"] = LOSes

In [23]:
def major_voting(labels):
    unique_labels = set(labels)
    count_labels = [labels.count(label) for label in unique_labels]

    sorted_labels = sorted(zip(unique_labels, count_labels), key=lambda x: x[1])
    if len(sorted_labels) > 1 and sorted_labels[0][1] == sorted_labels[1][1]:
        print("Oh no, many majors?")
    return sorted_labels[0][0]

def mean_voting(labels):
    l = ["A", "B", "C", "D", "E", "F"]
    values = {"A":0, "B":1, "C":2, "D":3, "E":4, "F":5}
    mean = sum(values[label] for label in labels) / len(labels)
    return l[min(round(mean), 5)]

In [24]:
compress_LOS = segment_status_df.groupby(by=["segment_id", "date", "weekday", "period"])["LOS"].apply(list)
compress_LOS = pd.DataFrame(compress_LOS).reset_index()
compress_LOS["LOS"] = compress_LOS["LOS"].apply(mean_voting)
compress_LOS

,segment_id,date,weekday,period,LOS
0,26,2021-04-16,4,period_0_30,A
1,33,2020-08-02,6,period_23_30,C
2,33,2020-08-03,0,period_0_00,D
3,67,2021-03-09,1,period_9_30,B
4,67,2021-03-23,1,period_9_30,B
...,...,...,...,...,...
33436,84529,2021-01-05,1,period_8_00,F
33437,84533,2021-01-09,5,period_13_30,F
33438,84533,2021-04-22,3,period_5_00,F
33439,84534,2021-01-09,5,period_13_30,D
